In [1]:
! jupyter nbconvert --to script google_bigquery.ipynb

[NbConvertApp] Converting notebook google_bigquery.ipynb to script
[NbConvertApp] Writing 2331 bytes to google_bigquery.py


In [2]:
def csv_to_bq(filename, dataset_id, table_id, overwrite=True, auto=True):
    """
    return : overwrite to : WRITE_APPEND if append
    """
    from google.cloud.bigquery import client
    from google.cloud import bigquery
    import os
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/simonprudhomme/Desktop/gcp_json_keys/maxi-price-1422fac585da.json'
    client = bigquery.Client()
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)
    
    job_config = bigquery.LoadJobConfig()
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.skip_leading_rows = 1
    if auto :
        job_config.autodetect = True
    else :
        job_config.schema = [
            bigquery.SchemaField("productSKU", "STRING"),
            bigquery.SchemaField("productName", "STRING"),
            bigquery.SchemaField("productPrice", "STRING"),
            bigquery.SchemaField("ratios", "STRING"),
            bigquery.SchemaField("date", "DATE"),
            bigquery.SchemaField("categories", "STRING")]
    if overwrite :
        job_config.write_disposition = 'WRITE_TRUNCATE'
    else :
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
    
    with open(filename, "rb") as source_file:
        job = client.load_table_from_file(source_file, table_ref, job_config=job_config)
    job.result()  # Waits for table load to complete.
    print("Loaded {} rows into {}:{}.".format(job.output_rows, dataset_id, table_id))
    return

In [3]:
def bq_to_csv(query):
    from google.cloud.bigquery import client
    from google.cloud import bigquery
    import os
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/simonprudhomme/Desktop/gcp_json_keys/maxi-price-1422fac585da.json'
    client = bigquery.Client()
    data = client.query(query).to_dataframe()
    print("Loaded DataFrame size {}".format(data.shape))
    return data 

In [4]:
# filename = '/Users/simonprudhomme/Desktop/dm/advertiser_expense_stats.csv'
# dataset_id = 'maxi_prices'
# table_id ='price'

# csv_to_bq(filename, dataset_id, table_id, overwrite=True,auto=True)

# query = """SELECT * FROM `maxi-price.maxi_prices.price`"""
# data = bq_to_csv(query)